In [42]:
import tensorflow.keras as keras
import numpy as np
from sklearn.utils import resample
import pandas as pd
import pickle
import os
import csv
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from statistics import mean
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from keras.regularizers import l2
MODEL_PATH = './model/'
DATA_PATH = './data/'

In [43]:
loc_data = pd.read_csv('data/bangkok', na_values=["?"], header=None)
target_dataset = loc_data.sample(n = 1200, replace = False)
df_rest = loc_data.loc[~loc_data.index.isin(target_dataset.index)]
shadow_dataset = df_rest.sample(n = 2000, replace = False)
df_rest = df_rest.loc[~df_rest.index.isin(shadow_dataset.index)]
attack_test_nonmembers = df_rest.sample(n = 600, replace = False)
attack_test_members =  target_dataset.iloc[:840,:].sample(n = 600, replace = False)

In [118]:
pur_data = np.load(DATA_PATH+'purchase100.npz')
features = pur_data['features']
labels = pur_data['labels']
data = pd.DataFrame(features[:,:])
labels = np.argmax(labels, axis=1)
data['600'] = labels

In [119]:
target_dataset = pd.DataFrame(data.iloc[1:20001,].values)
shadow_dataset = pd.DataFrame(data.iloc[15001:50001,].values)
attack_test_nonmembers = pd.DataFrame(data.iloc[75001:85001,].values)
attack_test_members = pd.DataFrame(data.iloc[5001:15001,].values)

In [ ]:
data = pd.read_csv('data/adult.data', na_values=["?"])
target_dataset = pd.DataFrame(data.iloc[1:10001,].values)
shadow_dataset = pd.DataFrame(data.iloc[15001:30001,].values)
attack_test_nonmembers = pd.DataFrame(data.iloc[11001:14001,].values)
attack_test_members = pd.DataFrame(data.iloc[2001:5001,].values)

In [ ]:
target_dataset = pd.read_csv('data/adult_sds.csv', na_values=["?"])
shadow_dataset = pd.read_csv('data/adultODS10K_to_25K.csv', na_values=["?"])
attack_test_nonmembers = pd.read_csv('data/adultODS25K_to_28K.csv', na_values=["?"])
attack_test_members = pd.read_csv('data/adultODS1_to_3K.csv', na_values=["?"])

In [44]:
def read_data(data_name):
    with np.load(DATA_PATH + data_name) as f:
        train_x, train_y, test_x, test_y = [f['arr_%d' % i] for i in range(len(f.files))]
    return train_x, train_y, test_x, test_y

In [45]:
def transform_data(dataset):
    x = dataset.iloc[:,1:dataset.shape[1]-1] # seperate the feature column from class label
    y = dataset.iloc[:,0] # label column

    dim=x.shape[1] # number of feature columns

    #num of classes
    num_classes=30

    for j in range(0,dim):
        if x.iloc[:,j].dtypes == object:   # transform categorical variables
            x.iloc[:,j] = x.iloc[:,j].astype('category') # change datatype to categorical
            x.iloc[:,j] = x.iloc[:,j].cat.codes # change from category name to category number
        else:  #transform numrical variables to standard scaler form 
            sc = StandardScaler()  
            val=np.array(x.iloc[:,j]).reshape(-1,1)
            std_data = sc.fit_transform(val)
            std_data = pd.DataFrame(std_data)
            x.iloc[:,j]=std_data

    y = y.astype('category') # change label to categorical
    y = y.cat.codes # change from category name to number

    x=np.array(x)
    y=np.array(y)
    
    
    y=to_categorical(y)
    
    return x, y, dim

In [84]:
def transform_location_data(dataset): 
    df_tot = dataset
    df_tot.dropna(inplace=True)

    trainX = df_tot.iloc[:,1:]
    trainY = df_tot.iloc[:,0]
    

    dim=trainX.shape[1]


    #num of classes
    num_classes=30

    trainX=np.array(trainX)
    trainY=np.array(trainY)
    
    
    trainY = to_categorical(trainY)

    return trainX, trainY, dim

In [47]:
def load_target_data(dataset, test_ratio):
    x, y, dim = transform_location_data(dataset)
    
    #trainX,testX, trainY, testY = train_test_split(x, y, test_size=test_ratio, random_state=0, stratify=y)
    trainX = x[0:840,]
    testX = x[840:,]
    trainY = y[0:840,]
    testY = y[840:,]
    return (trainX, trainY), (testX, testY), dim

In [48]:
def build_purchase_dnn(n_class,dim):
    model = Sequential()
    
    model.add(Dense(600, input_dim=dim))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    #model.add(Dense(1024), kernel_regularizer=l2(0.001))
    #model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(512, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))

    model.add(Dense(256, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(128, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    
    #opt = SGD(lr=0.01, momentum=0.9)
    #model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    act_layer=6
    
    return model, act_layer

In [49]:
def build_location_dnn(n_class,dim):
    model = Sequential()
    
    model.add(Dense(512, input_dim=dim, kernel_regularizer=l2(0.0007)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(248, kernel_regularizer=l2(0.0007)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(128, kernel_regularizer=l2(0.0007)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))

    model.add(Dense(64, kernel_regularizer=l2(0.0007)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
       
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    act_layer=6
    
    return model, act_layer

In [50]:
def build_simple_mlp(n_class,pix,d):

    model = Sequential()
    model.add(Dense(256, input_dim=pix))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(256, kernel_regularizer=l2(0.01)))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    
    #model.add(Dense(248))
    #model.add(Activation("relu"))
    #model.add(Dropout(0.01))

    model.add(Dense(64))
    model.add(Activation("relu"))
    model.add(Dropout(0.01))
    
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    act_layer=3
    
    return model, act_layer

In [51]:
def load_shadow_data(dataset, n_shadow, shadow_size, test_ratio):
    x, y, _ = transform_location_data(dataset)
    
    shadow_indices = np.arange(len(dataset))
    
   
    for i in range(n_shadow):
        shadow_i_indices = np.random.choice(shadow_indices, shadow_size, replace=False)
        shadow_i_x, shadow_i_y = x[shadow_i_indices], y[shadow_i_indices]
        trainX,testX, trainY, testY = train_test_split(shadow_i_x, shadow_i_y, test_size=test_ratio)
        print('shadow_i_trainX = ', len(trainX), 'shadow_i_trainY = ', len(trainY), 'shadow_i_testX = ', len(testX), 'shadow_i_testY = ', len(testY))
        
        np.savez(DATA_PATH + 'shadow_location{}_data.npz'.format(i), trainX, trainY, testX, testY)

In [52]:
def train_shadow_models(n_shadow, n_class, dim, channel):
    full_sm_train_pred=[]
    full_sm_train_class=[]
    
    full_sm_test_pred=[]
    full_sm_test_class=[]
    
    full_clz_train=[]
    full_clz_test=[]
    
    members=[]
    nonmembers=[]

    for j in range(n_shadow):
        
        print("Shadow Model ", j)
        
        print('Training shadow model {}'.format(j))
        data = read_data('shadow_location{}_data.npz'.format(j))
        x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test = data
        #print('x_shadow trian\n', x_shadow_train,'\n y_shadow trian\n', y_shadow_train, '\n x_shadow test\n', x_shadow_test, '\n y_shadow test\n', y_shadow_test)

        model, act_layer = build_location_dnn(n_class,dim)
            
            
        # fit model
        history = model.fit(x_shadow_train, y_shadow_train, epochs=EPS, batch_size=32, validation_data=(x_shadow_test, y_shadow_test), verbose=0)
    
        # evaluate model
        _, train_acc = model.evaluate(x_shadow_train, y_shadow_train, verbose=0)
        _, test_acc = model.evaluate(x_shadow_test, y_shadow_test, verbose=0)
        print("Shadow Train acc : ", (train_acc * 100.0),"Shadow Test acc : ", (test_acc * 100.0))

    
        #train SM
        sm_train_pred=model.predict(x_shadow_train, batch_size=32)
        sm_train_class=np.argmax(y_shadow_train,axis=1)
    
    
        #test SM
        sm_test_pred=model.predict(x_shadow_test, batch_size=32)
        sm_test_class=np.argmax(y_shadow_test,axis=1)
        
     
        full_sm_train_pred.append(sm_train_pred)        
        full_sm_train_class.append(sm_train_class)
        members.append(np.ones(len(sm_train_pred)))
        
        full_sm_test_pred.append(sm_test_pred)        
        full_sm_test_class.append(sm_test_class) 
        nonmembers.append(np.zeros(len(sm_test_pred)))


    full_sm_train_pred = np.vstack(full_sm_train_pred)
    full_sm_train_class = [item for sublist in full_sm_train_class for item in sublist]
    members = [item for sublist in members for item in sublist]
    
    full_sm_test_pred = np.vstack(full_sm_test_pred)
    full_sm_test_class = [item for sublist in full_sm_test_class for item in sublist]
    nonmembers = [item for sublist in nonmembers for item in sublist]
    
    shadow_train_performance=(full_sm_train_pred, np.array(full_sm_train_class))
    shadow_test_performance=(full_sm_test_pred, np.array(full_sm_test_class))


    ###atack data preparation
    attack_x = (full_sm_train_pred,full_sm_test_pred)
    #attack_x = np.vstack(attack_x)
    
    attack_y = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
    #attack_y = np.concatenate(attack_y)
    #attack_y = attack_y.astype('int32')
    
    
    classes = (np.array(full_sm_train_class),np.array(full_sm_test_class))
    #classes = np.array([item for sublist in classes for item in sublist])


    attack_dataset = (attack_x,attack_y,classes)

            
    return  shadow_train_performance, shadow_test_performance, attack_dataset, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, model

In [53]:
def define_attack_model(n_class):
    model = Sequential()
    
    model.add(Dense(1))
    model.add(Activation("relu"))
    
    model.add(Dense(1))
    model.add(Activation("relu"))

    model.add(Dense(n_class, activation='softmax'))

    # compile model
    opt = SGD(learning_rate=0.0001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [54]:
def attack_mlp(pix,d):

    model = Sequential()
    model.add(Dense(64, input_dim=pix))
    model.add(Activation("relu"))
    #model.add(Dropout(0.1))

#     model.add(Dense(32))
#     model.add(Activation("tanh"))
#     model.add(Dropout(0.01))
    
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    act_layer=1
    
    return model, act_layer

In [55]:
def prep_attack_data(n_attack_data):

    attack_mem = pd.DataFrame(n_attack_data[0][0])
    attack_nmem = pd.DataFrame(n_attack_data[0][1])
    
    attack_mem_status = pd.DataFrame(n_attack_data[1][0])
    attack_mem_status.columns = ["membership"]
    
    attack_nmem_status = pd.DataFrame(n_attack_data[1][1])
    attack_nmem_status.columns = ["membership"]
    
    real_class_mem = pd.DataFrame(n_attack_data[2][0])
    real_class_mem.columns = ["y"]
    
    real_class_nmem = pd.DataFrame(n_attack_data[2][1])
    real_class_nmem.columns = ["y"]

    memdf = pd.concat([attack_mem,attack_nmem],axis=0)
    memdf = memdf.reset_index(drop=True)

    memstatus =  pd.concat([attack_mem_status,attack_nmem_status],axis=0)
    memstatus = memstatus.reset_index(drop=True)

    realclass = pd.concat([real_class_mem,real_class_nmem],axis=0)
    realclass = realclass.reset_index(drop=True)

    attack_df = pd.concat([memdf,realclass,memstatus],axis=1)
    
    return attack_df

In [56]:
def prep_validation_data(attack_test_data):

    attack_mem = pd.DataFrame(attack_test_data[0][0])
    attack_nmem = pd.DataFrame(attack_test_data[0][1])
    
    attack_mem_status = pd.DataFrame(attack_test_data[1][0])
    attack_mem_status.columns = ["membership"]
    
    attack_nmem_status = pd.DataFrame(attack_test_data[1][1])
    attack_nmem_status.columns = ["membership"]
    
    real_class_mem = pd.DataFrame(attack_test_data[2][0])
    real_class_mem.columns = ["y"]
    
    real_class_nmem = pd.DataFrame(attack_test_data[2][1])
    real_class_nmem.columns = ["y"]
    
    mem_df = pd.concat([attack_mem,real_class_mem],axis=1)
    nmem_df = pd.concat([attack_nmem,real_class_nmem],axis=1)

#     memdf = pd.concat([attack_mem,attack_nmem],axis=0)
#     memdf = memdf.reset_index(drop=True)

#     memstatus =  pd.concat([attack_mem_status,attack_nmem_status],axis=0)
#     memstatus = memstatus.reset_index(drop=True)

#     realclass = pd.concat([real_class_mem,real_class_nmem],axis=0)
#     realclass = realclass.reset_index(drop=True)

#     attack_df = pd.concat([memdf,realclass,memstatus],axis=1)
    
    return mem_df, nmem_df

In [57]:
def load_attack_test_data(members, nonmembers):
    memberX, memberY, _ = transform_location_data(members)
    
    nonmemberX, nonmemberY, _ = transform_location_data(nonmembers)
    
    return memberX, memberY, nonmemberX, nonmemberY

In [58]:
def prety_print_result(mem, pred):
    tn, fp, fn, tp = confusion_matrix(mem, pred).ravel()
    print('TP: %d     FP: %d     FN: %d     TN: %d' % (tp, fp, fn, tn))
    if tp == fp == 0:
        print('PPV: 0\nAdvantage: 0')
    else:
        print('PPV: %.4f\nAdvantage: %.4f' % (tp / (tp + fp), tp / (tp + fn) - fp / (tn + fp)))

    return tp, fp, fn, tn, (tp / (tp + fp)), (tp / (tp + fn) - fp / (tn + fp)), ((tp+tn)/(tp+tn+fp+fn)),  (tp / (tp + fn))

In [59]:
def train_attack_model(attack_data, check_membership, n_hidden=50, learning_rate=0.01, batch_size=200, epochs=50, model='nn', l2_ratio=1e-7):

    x, y,  classes = attack_data

    train_x = x[0]
    train_y = y[0]
    test_x = x[1]
    test_y = y[1]
    train_classes = classes[0]
    test_classes = classes[1]
    
    
    checkmem_prediction_vals, checkmem_membership_status, checkmem_class_status = check_membership
    
    checkmem_prediction_vals=np.vstack(checkmem_prediction_vals)
    checkmem_membership_status=np.array([item for sublist in checkmem_membership_status for item in sublist])
    checkmem_class_status=np.array([item for sublist in checkmem_class_status for item in sublist])
    
    train_indices = np.arange(len(train_x))
    test_indices = np.arange(len(test_x))
    unique_classes = np.unique(train_classes)


    predicted_membership, target_membership = [], []
    for c in unique_classes:
        print("Class : ", c)
        c_train_indices = train_indices[train_classes == c]
        c_train_x, c_train_y = train_x[c_train_indices], train_y[c_train_indices]
        c_test_indices = test_indices[test_classes == c]
        c_test_x, c_test_y = test_x[c_test_indices], test_y[c_test_indices]
        c_dataset = (c_train_x, c_train_y, c_test_x, c_test_y)        
        
        full_cx_data=(c_train_x,c_test_x)
        full_cx_data = np.vstack(full_cx_data)

        full_cy_data=(c_train_y,c_test_y)
        full_cy_data = np.array([item for sublist in full_cy_data for item in sublist])
        
#         over_sampler = SMOTE(k_neighbors=2)
#         full_cx_data, full_cy_data = over_sampler.fit_resample(full_cx_data, full_cy_data)
#         full_cy_data = to_categorical(full_cy_data)
              
        
#         classifier = define_attack_model(2)
#         history = classifier.fit(full_cx_data, full_cy_data, epochs=EPS, batch_size=32, verbose=0)
        
        d=1
        pix = full_cx_data.shape[1]
        classifier, _ = attack_mlp(pix,d)
        history = classifier.fit(full_cx_data, full_cy_data, epochs=EPS, batch_size=32, verbose=0)
        

        #get predictions on real train and test data
        c_indices = np.where(checkmem_class_status==c)
        pred_y = classifier.predict(checkmem_prediction_vals[c_indices])
        c_pred_y = np.argmax(pred_y, axis=1)
        c_target_y = checkmem_membership_status[c_indices]
        
       
        target_membership.append(c_target_y)
        predicted_membership.append(c_pred_y)

    target_membership=np.array([item for sublist in target_membership for item in sublist])
    predicted_membership=np.array([item for sublist in predicted_membership for item in sublist])


    tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (target_membership,predicted_membership)   
    return tp, fp, fn, tn, precision, advj, acc, recall

In [60]:
def shokri_attack(attack_df, mem_validation, nmem_validation):
    
    predicted_membership, predicted_nmembership, true_membership, TP_idx, TN_idx  = [], [], [], [], []

    class_val = np.unique(attack_df['y'])
    ncval=attack_df.shape[1]-1
    
    for c_val in class_val:

        print(c_val)
        
        filter_rec_all = attack_df[(attack_df['y'] == c_val)]
        filter_rec_idx = np.array(filter_rec_all.index)
        
        attack_feat = filter_rec_all.iloc[:, 0:ncval]
        attack_class = filter_rec_all['membership']
             
        d=1
        pix = attack_feat.shape[1]
        
        attack_model, _ = attack_mlp(pix,d)
        
       
        history = attack_model.fit(attack_feat, attack_class, epochs=EPS, batch_size=32, verbose=0)
        
        mcval=mem_validation.shape[1]-1
        
        
        check_mem_feat = mem_validation[mem_validation['y']==c_val]
        check_nmem_feat = nmem_validation[nmem_validation['y']==c_val]
        
        if (len(check_mem_feat)!=0) and (len(check_nmem_feat)!=0):
        
            check_mem_feat_idx =  np.array(check_mem_feat.index)


            check_nmem_feat_idx =  np.array(check_nmem_feat.index)

            #print(check_nmem_feat_idx)
            #print(np.argmax(mpred,axis=1)==0)


            mpred = attack_model.predict(np.array(check_mem_feat))    
            predicted_membership.append(np.argmax(mpred,axis=1) )

            nmpred = attack_model.predict(np.array(check_nmem_feat))    
            predicted_nmembership.append(np.argmax(nmpred,axis=1) )        



            TP_idx.append(check_mem_feat_idx[np.where(np.argmax(mpred,axis=1)==1)[0]])

            TN_idx.append(check_nmem_feat_idx[np.where(np.argmax(nmpred,axis=1)==0)[0]])

    pred_members = np.array([item for sublist in predicted_membership for item in sublist])
    pred_nonmembers = np.array([item for sublist in predicted_nmembership for item in sublist])
    
    TP_idx_list = np.array([item for sublist in TP_idx for item in sublist])
    TN_idx_list = np.array([item for sublist in TN_idx for item in sublist])
    
    members=np.array(list(pred_members))
    nonmembers=np.array(list(pred_nonmembers))
    
    pred_membership = np.concatenate([members,nonmembers])
    ori_membership = np.concatenate([np.ones(len(members)), np.zeros(len(nonmembers))])
    
    return pred_membership, ori_membership, TP_idx_list, TN_idx_list

In [85]:
# trian target model
per_class_sample=40
channel=1   
EPS=100
act_layer=6
n_class = 31
VERBOSE = 0
test_ratio = 0.3

(target_trainX, target_trainY), (target_testX, target_testY), dim = load_target_data(target_dataset, test_ratio)
target_model,_ = build_location_dnn(n_class,dim)
#get_trained_keras_models(model, (target_trainX, target_trainY), (target_testX, target_testY), num_models=1)
history = target_model.fit(target_trainX, target_trainY, epochs=EPS, batch_size=32, verbose=VERBOSE)
score = target_model.evaluate(target_testX, target_testY, verbose=VERBOSE)
print('\n', 'Model test accuracy:', score[1])


 Model test accuracy: 0.5305555462837219


In [86]:
# prepare attack test data

members = []
nonmembers = []

memberX, memberY, nonmemberX, nonmemberY = load_attack_test_data(attack_test_members, attack_test_nonmembers)

# member
target_model_member_pred = target_model.predict(memberX, batch_size=32)
target_model_member_class = np.argmax(memberY, axis=1)
target_model_member_pred = np.vstack(target_model_member_pred)
#target_model_member_class = [item for sublist in target_model_member_class for item in sublist]
members.append(np.ones(len(target_model_member_pred)))
members = [item for sublist in members for item in sublist]


# nonmember
target_model_nonmember_pred = target_model.predict(nonmemberX, batch_size=32)
target_model_nonmember_class = np.argmax(nonmemberY, axis=1)
target_model_nonmember_pred = np.vstack(target_model_nonmember_pred)
#target_model_nonmember_class = [item for sublist in target_model_nonmember_class for item in sublist]
nonmembers.append(np.zeros(len(target_model_nonmember_pred)))
nonmembers = [item for sublist in nonmembers for item in sublist]

full_attack_test_pred_val = (target_model_member_pred, target_model_nonmember_pred)
full_attack_test_mem_status = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
full_attack_test_class_status = (np.array(target_model_member_class),np.array(target_model_nonmember_class))

print('\n pred', full_attack_test_pred_val)
print('\n class', full_attack_test_class_status)
print('\n mem status', full_attack_test_mem_status)

attack_test_data = (full_attack_test_pred_val, full_attack_test_mem_status,full_attack_test_class_status)

19/19 [==============================] - 0s 3ms/step

 pred (array([[2.2451704e-05, 2.8579988e-04, 3.1301269e-04, ..., 7.5937379e-08,
        5.3171651e-05, 1.3372452e-06],
       [9.2460992e-05, 4.7596907e-03, 8.2041064e-05, ..., 4.2755586e-05,
        1.2596275e-04, 1.0706383e-05],
       [6.7628163e-05, 9.4923557e-04, 9.8687476e-01, ..., 1.6501830e-09,
        9.8220815e-05, 5.1238608e-06],
       ...,
       [1.3854070e-05, 1.7114902e-05, 8.9391642e-06, ..., 7.1200309e-05,
        4.5617655e-04, 3.6587691e-04],
       [2.7609134e-05, 8.3431747e-04, 1.7158538e-05, ..., 4.8328440e-07,
        1.7086437e-04, 2.7603130e-07],
       [8.9783662e-06, 3.7671265e-04, 1.0525980e-03, ..., 7.0322119e-07,
        4.5053280e-06, 1.4845864e-06]], dtype=float32), array([[3.0896984e-04, 3.0560547e-04, 9.0791506e-01, ..., 1.0290515e-06,
        9.4257668e-04, 7.9719713e-03],
       [7.3096761e-04, 1.2062288e-05, 1.6828367e-04, ..., 5.4822018e-04,
        2.8696761e-01, 9.8327138e-02],
       [2.2453

In [278]:
# save attack test dataset to csv
df = pd.DataFrame()
mem = pd.Series(full_attack_test_pred_val[0][:,0])
nonmem = pd.Series(full_attack_test_pred_val[1][:,0])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
len(total)
df['prob_class_0'] = total
df = df.reset_index()
mem = pd.Series(full_attack_test_pred_val[0][:,1])
nonmem = pd.Series(full_attack_test_pred_val[1][:,1])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
df['prob_class 1'] = total

mem = pd.Series(full_attack_test_mem_status[0][:])
nonmem = pd.Series(full_attack_test_mem_status[1][:])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
df['mem_status'] = total

mem = pd.Series(full_attack_test_class_status[0][:])
nonmem = pd.Series(full_attack_test_class_status[1][:])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
df['class_status'] = total

df.drop("index", axis=1, inplace=True)


df.to_csv('attack_test_data.csv')

In [87]:
#prepare shadow dataset
n_shadow_models = 60
shadow_data_size = 1200
test_ratio = 0.3

load_shadow_data(shadow_dataset, n_shadow_models, shadow_data_size, test_ratio)

shadow_i_trainX =  840 shadow_i_trainY =  840 shadow_i_testX =  360 shadow_i_testY =  360
shadow_i_trainX =  840 shadow_i_trainY =  840 shadow_i_testX =  360 shadow_i_testY =  360
shadow_i_trainX =  840 shadow_i_trainY =  840 shadow_i_testX =  360 shadow_i_testY =  360
shadow_i_trainX =  840 shadow_i_trainY =  840 shadow_i_testX =  360 shadow_i_testY =  360
shadow_i_trainX =  840 shadow_i_trainY =  840 shadow_i_testX =  360 shadow_i_testY =  360
shadow_i_trainX =  840 shadow_i_trainY =  840 shadow_i_testX =  360 shadow_i_testY =  360
shadow_i_trainX =  840 shadow_i_trainY =  840 shadow_i_testX =  360 shadow_i_testY =  360
shadow_i_trainX =  840 shadow_i_trainY =  840 shadow_i_testX =  360 shadow_i_testY =  360
shadow_i_trainX =  840 shadow_i_trainY =  840 shadow_i_testX =  360 shadow_i_testY =  360
shadow_i_trainX =  840 shadow_i_trainY =  840 shadow_i_testX =  360 shadow_i_testY =  360
shadow_i_trainX =  840 shadow_i_trainY =  840 shadow_i_testX =  360 shadow_i_testY =  360
shadow_i_t

In [88]:
#train shadow model
per_class_sample=40
channel=1
EPS=100
act_layer=6
n_class = 31
VERBOSE = 0
test_ratio = 0.3

n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init = train_shadow_models(n_shadow_models, n_class, dim, channel)

Shadow Model  0
Training shadow model 0
Shadow Train acc :  99.88095164299011 Shadow Test acc :  42.222222685813904
12/12 [==============================] - 0s 2ms/step
Shadow Model  1
Training shadow model 1
Shadow Train acc :  100.0 Shadow Test acc :  47.22222089767456
12/12 [==============================] - 0s 4ms/step
Shadow Model  2
Training shadow model 2
Shadow Train acc :  100.0 Shadow Test acc :  48.055556416511536
12/12 [==============================] - 0s 2ms/step
Shadow Model  3
Training shadow model 3
Shadow Train acc :  100.0 Shadow Test acc :  47.22222089767456
12/12 [==============================] - 0s 2ms/step
Shadow Model  4
Training shadow model 4
Shadow Train acc :  79.16666865348816 Shadow Test acc :  34.44444537162781
12/12 [==============================] - 0s 2ms/step
Shadow Model  5
Training shadow model 5
Shadow Train acc :  100.0 Shadow Test acc :  53.05555462837219
12/12 [==============================] - 0s 2ms/step
Shadow Model  6
Training shadow model 

12/12 [==============================] - 0s 8ms/step
Shadow Model  39
Training shadow model 39
Shadow Train acc :  100.0 Shadow Test acc :  51.11111402511597
12/12 [==============================] - 0s 2ms/step
Shadow Model  40
Training shadow model 40
Shadow Train acc :  99.52380657196045 Shadow Test acc :  46.11110985279083
12/12 [==============================] - 0s 2ms/step
Shadow Model  41
Training shadow model 41
Shadow Train acc :  100.0 Shadow Test acc :  51.38888955116272
12/12 [==============================] - 0s 2ms/step
Shadow Model  42
Training shadow model 42
Shadow Train acc :  100.0 Shadow Test acc :  48.88888895511627
12/12 [==============================] - 0s 3ms/step
Shadow Model  43
Training shadow model 43
Shadow Train acc :  100.0 Shadow Test acc :  47.49999940395355
12/12 [==============================] - 0s 2ms/step
Shadow Model  44
Training shadow model 44
Shadow Train acc :  100.0 Shadow Test acc :  48.61111044883728
12/12 [==============================] -

In [90]:
attack_df = prep_attack_data(n_attack_data)
attack_df

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])

In [91]:
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
nmem_validation

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,y
0,0.000309,0.000306,0.907915,0.001118,0.000593,0.014152,0.002854,1.307487e-06,0.000084,2.756053e-04,...,4.222034e-03,3.473059e-04,0.000090,0.000016,1.056595e-02,0.000462,1.029052e-06,0.000943,7.971971e-03,2
1,0.000731,0.000012,0.000168,0.001460,0.032700,0.001204,0.000010,6.073690e-05,0.000673,2.763821e-04,...,4.686335e-01,1.639225e-02,0.004764,0.002624,5.966214e-02,0.016756,5.482202e-04,0.286968,9.832714e-02,29
2,0.000225,0.604389,0.000290,0.000320,0.000134,0.009279,0.046155,1.814771e-01,0.120881,9.508774e-05,...,3.024930e-06,1.654286e-04,0.015224,0.003167,5.620118e-06,0.000180,1.767857e-04,0.000100,4.197839e-07,1
3,0.000340,0.003887,0.000002,0.000014,0.000038,0.000010,0.000049,3.199356e-04,0.000245,6.803751e-03,...,3.868211e-02,3.358580e-01,0.002763,0.000012,4.919092e-04,0.000102,1.359883e-01,0.000312,1.293991e-03,23
4,0.000133,0.122700,0.000234,0.000115,0.000145,0.005945,0.005737,1.291345e-04,0.002174,3.152572e-05,...,7.577279e-04,6.375201e-05,0.820674,0.000081,9.395346e-06,0.000006,4.940748e-05,0.000006,2.446454e-05,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.001992,0.000828,0.006391,0.005343,0.001124,0.004704,0.003768,3.661131e-03,0.028536,7.559169e-02,...,2.011770e-03,4.047854e-04,0.000545,0.072970,6.815073e-03,0.000046,5.415375e-04,0.000228,5.823603e-04,21
596,0.000354,0.000607,0.000029,0.000133,0.000117,0.000005,0.000374,3.596262e-03,0.000128,2.172195e-02,...,4.109001e-03,2.503201e-01,0.000102,0.000307,1.208794e-04,0.000004,5.958360e-01,0.000256,4.241396e-03,23
597,0.000007,0.000608,0.000026,0.000021,0.000018,0.000629,0.000696,5.465820e-06,0.997620,4.959995e-07,...,2.802638e-07,7.759500e-06,0.000032,0.000001,2.423435e-07,0.000012,8.654787e-08,0.000008,2.753220e-07,8
598,0.000378,0.000090,0.416962,0.000428,0.002955,0.064196,0.008082,6.978997e-07,0.000415,9.965130e-05,...,6.928544e-03,7.730046e-04,0.002578,0.000901,1.955544e-04,0.000123,6.303369e-05,0.000532,1.521364e-01,2


In [92]:
pred_membership, ori_membership, TP_idx_list, TN_idx_list = shokri_attack(attack_df, mem_validation, nmem_validation)

1
1/1 [==============================] - 0s 32ms/step
2
1/1 [==============================] - 0s 30ms/step
3
1/1 [==============================] - 0s 28ms/step
4
1/1 [==============================] - 0s 30ms/step
5
1/1 [==============================] - 0s 25ms/step
6
1/1 [==============================] - 0s 31ms/step
7
1/1 [==============================] - 0s 28ms/step
8
2/2 [==============================] - 0s 3ms/step
9
1/1 [==============================] - 0s 23ms/step
10
1/1 [==============================] - 0s 27ms/step
11
1/1 [==============================] - 0s 30ms/step
12
1/1 [==============================] - 0s 34ms/step
13
1/1 [==============================] - 0s 29ms/step
14
1/1 [==============================] - 0s 29ms/step
15
1/1 [==============================] - 0s 28ms/step
16
1/1 [==============================] - 0s 28ms/step
17
1/1 [==============================] - 0s 29ms/step
18
1/1 [==============================] - 0s 26ms/step
19
1/1 [============

In [93]:
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)  

TP: 600     FP: 148     FN: 0     TN: 452
PPV: 0.8021
Advantage: 0.7533


In [94]:
precision

0.8021390374331551

In [282]:
#save the prepared attack data on disk
np.savez(DATA_PATH + 'attack_purchase_data.npz', n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init)

/home/msnkhan/anaconda3/envs/r4-base/lib/python3.7/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


INFO:tensorflow:Assets written to: ram://a9382bc7-fd48-42a6-8436-af2bd2196281/assets


In [ ]:
# read stored attack model training data 
data_name = 'attack_adult_data.npz'
with np.load(DATA_PATH + data_name, allow_pickle=True) as f:
        n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init = [f['arr_%d' % i] for i in range(len(f.files))]

In [140]:
tp, fp, fn, tn, precision, advj, acc, recall = train_attack_model(n_attack_data, attack_test_data)
#target_membership, predicted_membership = train_attack_model(n_attack_data, attack_test_data)

Class :  0
5/5 [==============================] - 0s 2ms/step
Class :  1
4/4 [==============================] - 0s 2ms/step
Class :  2
16/16 [==============================] - 0s 1ms/step
Class :  3
4/4 [==============================] - 0s 2ms/step
Class :  4
10/10 [==============================] - 0s 1ms/step
Class :  5
6/6 [==============================] - 0s 2ms/step
Class :  6
8/8 [==============================] - 0s 1ms/step
Class :  7
7/7 [==============================] - 0s 2ms/step
Class :  8
6/6 [==============================] - 0s 2ms/step
Class :  9
6/6 [==============================] - 0s 2ms/step
Class :  10
10/10 [==============================] - 0s 2ms/step
Class :  11
6/6 [==============================] - 0s 2ms/step
Class :  12
7/7 [==============================] - 0s 2ms/step
Class :  13
8/8 [==============================] - 0s 2ms/step
Class :  14
5/5 [==============================] - 0s 2ms/step
Class :  15
16/16 [==============================] - 0s 1ms

In [284]:
precision

0.5193872315473136

In [ ]:
target_membership

In [ ]:
predicted_membership

In [ ]:
df = pd.DataFrame()
df['target'] = pd.Series(target_membership)
df['predicted'] = pd.Series(predicted_membership)

In [ ]:
df

In [ ]:
df.to_csv('test.csv')

In [ ]:
attack_data = n_attack_data
check_membership = attack_test_data
n_hidden=50
learning_rate=0.01
batch_size=200
epochs=50
model='nn'
l2_ratio=1e-7

In [ ]:
x, y,  classes = attack_data

train_x = x[0]
train_y = y[0]
test_x = x[1]
test_y = y[1]
train_classes = classes[0]
test_classes = classes[1]

#print(tra)


In [ ]:
checkmem_prediction_vals, checkmem_membership_status, checkmem_class_status = check_membership

checkmem_prediction_vals=np.vstack(checkmem_prediction_vals)
checkmem_membership_status=np.array([item for sublist in checkmem_membership_status for item in sublist])
checkmem_class_status=np.array([item for sublist in checkmem_class_status for item in sublist])

train_indices = np.arange(len(train_x))
test_indices = np.arange(len(test_x))
unique_classes = np.unique(train_classes)


predicted_membership, target_membership = [], []

In [ ]:
for c in unique_classes:
    print("Class : ", c)
    c_train_indices = train_indices[train_classes == c]
    c_train_x, c_train_y = train_x[c_train_indices], train_y[c_train_indices]
    c_test_indices = test_indices[test_classes == c]
    c_test_x, c_test_y = test_x[c_test_indices], test_y[c_test_indices]
    c_dataset = (c_train_x, c_train_y, c_test_x, c_test_y)  

    full_cx_data=(c_train_x,c_test_x)
    full_cx_data = np.vstack(full_cx_data)

    full_cy_data=(c_train_y,c_test_y)
    full_cy_data = np.array([item for sublist in full_cy_data for item in sublist])
    full_cy_data = to_categorical(full_cy_data)

    classifier = define_attack_model(2)
    history = classifier.fit(full_cx_data, full_cy_data, epochs=200, batch_size=32, verbose=0)
    #classifier.save('model/attack_model_class{}'.format(c))

    #get predictions on real train and test data
    c_indices = np.where(checkmem_class_status==c)
    predict_y = classifier.predict(checkmem_prediction_vals[c_indices])
    print(predict_y)
    c_pred_y = np.argmax(classifier.predict(checkmem_prediction_vals[c_indices]),axis=1)
    #c_pred_y = np.where(predict_y > 0.5, 1,0)
    #c_pred_y = classifier.predict_classes(checkmem_prediction_vals[c_indices])

    c_target_y = checkmem_membership_status[c_indices]


    target_membership.append(c_target_y)
    predicted_membership.append(c_pred_y)




In [ ]:
c_indices = np.where(checkmem_class_status==c)
predict_y = classifier.predict(checkmem_prediction_vals[c_indices], batch_size=32)
c_pred_y = np.where(predict_y > 0.5, 1,0)
c_target_y = checkmem_membership_status[c_indices]
target_membership.append(c_target_y)
predicted_membership.append(c_pred_y)

In [ ]:
target_membership=np.array([item for sublist in target_membership for item in sublist])
predicted_membership=np.array([item for sublist in predicted_membership for item in sublist])

In [ ]:
prety_print_result (target_membership,predicted_membership)

In [ ]:
np.vstack(predicted_membership).shape

In [ ]:
df = pd.DataFrame()
df['target'] = pd.Series(target_membership)
df['predicted'] = pd.Series(predicted_membership.reshape((len(predicted_membership))))

In [ ]:
df

In [ ]:
predicted_membership